# Day 2, Session 3: Smart Routing & Optimization

## From Concurrent Multimodal to Intelligent Decision Making

Sessions 1 & 2 gave us concurrent processing and multimodal intelligence. Now we add **smart routing** - the ability to make intelligent decisions about how to process documents based on their complexity, cost constraints, and performance requirements.

This represents the evolution from "processing everything the same way" to **adaptive, cost-aware AI systems**.

### What We're Building

An intelligent document routing system that:
1. **Analyzes Document Complexity** automatically (simple, moderate, complex)
2. **Routes to Optimal Processing Path** (fast/cheap vs accurate/expensive)
3. **Balances Cost vs Quality** based on business requirements
4. **Adapts in Real-Time** to system load and performance metrics
5. **Optimizes Resource Usage** across the entire processing pipeline

This bridges **intelligent processing** with **business optimization** - the key to production AI systems.

**Duration: 25 minutes**

In [ ]:
# Environment setup - Building on Sessions 1 & 2
import os
from dotenv import load_dotenv
load_dotenv()

# Core libraries from previous sessions
from concurrent.futures import ThreadPoolExecutor
import asyncio
import time
import threading
import json
from typing import Dict, List, Optional, Any, TypedDict, Union
from enum import Enum
from dataclasses import dataclass
from datetime import datetime
import random
import psutil

# LLM server configuration
OLLAMA_URL = os.getenv('OLLAMA_URL', 'http://XX.XX.XX.XX')
OLLAMA_API_TOKEN = os.getenv('OLLAMA_API_TOKEN', 'YOUR_TOKEN_HERE')
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'qwen3:8b')

print("🧠 Smart Routing & Optimization Setup")
print(f"   🔗 LLM Server: {'✅ Configured' if OLLAMA_URL != 'http://XX.XX.XX.XX' else '❌ Mock mode'}")
print(f"   ⚡ Session 1: Concurrent processing foundation")
print(f"   🎭 Session 2: Multimodal intelligence capabilities")
print(f"   🧠 Session 3: Smart routing and cost optimization")

In [ ]:
# Install required packages
!pip install -q requests python-dotenv
!pip install -q langgraph pydantic
!pip install -q psutil

In [ ]:
# Import LangGraph for intelligent routing
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
import requests

# Performance tracking from Session 1
class PerformanceTracker:
    """Enhanced performance tracking for routing decisions"""
    
    def __init__(self):
        self.timings = {}
        self.costs = {}
        self.quality_scores = {}
        self.routing_decisions = {}
        self.lock = threading.Lock()
    
    def start_operation(self, operation_id: str, route_type: str, estimated_cost: float):
        """Start tracking an operation with routing context"""
        with self.lock:
            self.timings[operation_id] = {
                'start': time.time(),
                'route_type': route_type,
                'estimated_cost': estimated_cost
            }
            self.routing_decisions[operation_id] = route_type
    
    def end_operation(self, operation_id: str, quality_score: float = 0.0) -> float:
        """End tracking and calculate metrics"""
        with self.lock:
            if operation_id in self.timings:
                duration = time.time() - self.timings[operation_id]['start']
                route_type = self.timings[operation_id]['route_type']
                estimated_cost = self.timings[operation_id]['estimated_cost']
                
                # Calculate actual cost based on duration and route type
                cost_per_second = {'fast': 0.1, 'balanced': 0.3, 'accurate': 0.8}.get(route_type, 0.3)
                actual_cost = duration * cost_per_second
                
                self.timings[operation_id].update({
                    'duration': duration,
                    'actual_cost': actual_cost
                })
                
                self.costs[operation_id] = actual_cost
                self.quality_scores[operation_id] = quality_score
                
                return duration
            return 0.0
    
    def get_routing_analytics(self) -> Dict[str, Any]:
        """Get analytics for routing optimization"""
        with self.lock:
            if not self.timings:
                return {'status': 'no_data'}
            
            route_stats = {}
            for op_id, timing in self.timings.items():
                route_type = timing['route_type']
                if route_type not in route_stats:
                    route_stats[route_type] = {
                        'count': 0,
                        'total_time': 0,
                        'total_cost': 0,
                        'total_quality': 0
                    }
                
                route_stats[route_type]['count'] += 1
                route_stats[route_type]['total_time'] += timing.get('duration', 0)
                route_stats[route_type]['total_cost'] += self.costs.get(op_id, 0)
                route_stats[route_type]['total_quality'] += self.quality_scores.get(op_id, 0)
            
            # Calculate averages
            for route_type, stats in route_stats.items():
                count = stats['count']
                stats.update({
                    'avg_time': stats['total_time'] / count,
                    'avg_cost': stats['total_cost'] / count,
                    'avg_quality': stats['total_quality'] / count,
                    'efficiency': stats['total_quality'] / max(stats['total_cost'], 0.01)
                })
            
            return {
                'route_stats': route_stats,
                'total_operations': len(self.timings),
                'total_cost': sum(self.costs.values()),
                'avg_quality': sum(self.quality_scores.values()) / len(self.quality_scores) if self.quality_scores else 0
            }

# Global tracker for routing optimization
routing_tracker = PerformanceTracker()

def call_llm(prompt, model=DEFAULT_MODEL):
    """Call LLM with error handling"""
    headers = {
        "Authorization": f"Bearer {OLLAMA_API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    data = {"model": model, "prompt": prompt}
    
    try:
        response = requests.post(f"{OLLAMA_URL}/think", headers=headers, json=data)
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

def get_memory_usage():
    """Get current memory usage"""
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # MB

print("📊 Enhanced performance tracking ready for smart routing")
print("🔍 Will track: timing, cost, quality, and routing efficiency")

## Step 1: Document Complexity Analysis

Building on Session 2's multimodal intelligence, we analyze document complexity to make smart routing decisions.

In [ ]:
# Document complexity classification
class ComplexityLevel(Enum):
    SIMPLE = "simple"        # Standard invoices, clear structure
    MODERATE = "moderate"    # Multi-page, some complexity
    COMPLEX = "complex"      # Poor quality, unusual format, multi-language
    CRITICAL = "critical"    # High-value, requires maximum accuracy

class ProcessingRoute(Enum):
    FAST = "fast"           # Quick, basic processing (OCR + simple extraction)
    BALANCED = "balanced"   # Standard processing (OCR + LLM analysis)
    ACCURATE = "accurate"   # Deep processing (Multi-modal + validation)
    PREMIUM = "premium"     # Human-in-the-loop for critical documents

class DocumentAnalysis(BaseModel):
    """Analysis results for routing decisions"""
    document_id: str
    complexity: ComplexityLevel
    confidence: float = Field(ge=0.0, le=1.0)
    
    # Complexity factors
    text_quality: float = Field(ge=0.0, le=1.0, description="OCR confidence")
    structure_clarity: float = Field(ge=0.0, le=1.0, description="Layout clarity")
    content_complexity: float = Field(ge=0.0, le=1.0, description="Information complexity")
    
    # Business factors
    monetary_value: float = Field(ge=0.0, description="Invoice amount")
    vendor_trust_level: float = Field(ge=0.0, le=1.0, description="Vendor reliability")
    
    # Recommended routing
    recommended_route: ProcessingRoute
    estimated_cost: float
    estimated_time: float
    expected_quality: float = Field(ge=0.0, le=1.0)

# Smart routing state (extends Session 2's multimodal state)
class SmartRoutingState(TypedDict):
    # Document information
    document_id: str
    document_content: Dict[str, Any]  # Text, images, metadata
    
    # Analysis results
    complexity_analysis: Optional[DocumentAnalysis]
    
    # Routing decision
    selected_route: Optional[ProcessingRoute]
    route_reason: Optional[str]
    
    # Processing results
    processing_results: Optional[Dict[str, Any]]
    quality_score: Optional[float]
    
    # Performance metrics
    actual_cost: Optional[float]
    actual_time: Optional[float]
    efficiency_score: Optional[float]
    
    # System context
    current_load: float  # System load factor
    cost_budget: float   # Available budget
    quality_requirement: float  # Minimum quality needed

def analyze_document_complexity(state: SmartRoutingState) -> SmartRoutingState:
    """Analyze document complexity for intelligent routing"""
    doc_id = state['document_id']
    content = state['document_content']
    
    print(f"🔍 Analyzing complexity for {doc_id}...")
    
    # Simulate multimodal analysis (building on Session 2)
    text_content = content.get('text', '')
    image_quality = content.get('image_quality', 0.8)
    page_count = content.get('page_count', 1)
    amount = content.get('amount', 1000.0)
    vendor = content.get('vendor', 'Unknown')
    
    # Calculate complexity factors
    text_quality = min(1.0, len(text_content) / 200.0) * image_quality  # Quality based on content and image
    structure_clarity = max(0.3, 1.0 - (page_count - 1) * 0.2)  # Decreases with page count
    content_complexity = min(1.0, len(text_content.split()) / 100.0)  # Based on content length
    
    # Business factors
    vendor_trust = 0.9 if 'trusted' in vendor.lower() else 0.7 if 'corp' in vendor.lower() else 0.5
    
    # Determine complexity level
    complexity_score = (text_quality + structure_clarity + content_complexity) / 3
    
    if amount > 10000 or complexity_score < 0.4:
        complexity = ComplexityLevel.COMPLEX
    elif amount > 5000 or complexity_score < 0.6:
        complexity = ComplexityLevel.MODERATE
    else:
        complexity = ComplexityLevel.SIMPLE
    
    # High-value documents get critical classification
    if amount > 50000:
        complexity = ComplexityLevel.CRITICAL
    
    # Calculate routing recommendation
    route, cost, time, quality = _recommend_processing_route(
        complexity, amount, vendor_trust, state['quality_requirement'], state['cost_budget']
    )
    
    analysis = DocumentAnalysis(
        document_id=doc_id,
        complexity=complexity,
        confidence=min(0.95, complexity_score + 0.1),
        text_quality=text_quality,
        structure_clarity=structure_clarity,
        content_complexity=content_complexity,
        monetary_value=amount,
        vendor_trust_level=vendor_trust,
        recommended_route=route,
        estimated_cost=cost,
        estimated_time=time,
        expected_quality=quality
    )
    
    state['complexity_analysis'] = analysis
    
    print(f"   📊 Complexity: {complexity.value} (confidence: {analysis.confidence:.2f})")
    print(f"   💰 Amount: ${amount:,.2f} | Vendor trust: {vendor_trust:.2f}")
    print(f"   🎯 Recommended route: {route.value} (cost: ${cost:.2f}, time: {time:.1f}s, quality: {quality:.2f})")
    
    return state

def _recommend_processing_route(complexity: ComplexityLevel, amount: float, vendor_trust: float, 
                               quality_req: float, budget: float) -> tuple:
    """Recommend processing route based on multiple factors"""
    
    # Base routing logic
    if complexity == ComplexityLevel.CRITICAL or amount > 25000:
        return ProcessingRoute.PREMIUM, 5.0, 30.0, 0.98
    elif complexity == ComplexityLevel.COMPLEX or (amount > 10000 and vendor_trust < 0.7):
        return ProcessingRoute.ACCURATE, 2.5, 15.0, 0.95
    elif complexity == ComplexityLevel.MODERATE or quality_req > 0.8:
        return ProcessingRoute.BALANCED, 1.0, 8.0, 0.88
    else:
        return ProcessingRoute.FAST, 0.3, 3.0, 0.75

print("🧠 Document complexity analysis ready")
print("🔍 Analyzes: text quality, structure, content, business factors")
print("🎯 Outputs: complexity level + optimal processing route recommendation")

## Step 2: Intelligent Routing Engine

The routing engine makes decisions based on document analysis, system load, and business constraints.

In [ ]:
class RoutingEngine:
    """Intelligent routing engine with dynamic optimization"""
    
    def __init__(self):
        self.route_performance = {route: [] for route in ProcessingRoute}
        self.system_load_threshold = 0.8
        self.cost_optimization_factor = 1.0
        self.quality_priority_factor = 1.0
    
    def route_document(self, state: SmartRoutingState) -> SmartRoutingState:
        """Make intelligent routing decision"""
        analysis = state['complexity_analysis']
        current_load = state['current_load']
        cost_budget = state['cost_budget']
        quality_req = state['quality_requirement']
        
        print(f"🧠 Routing decision for {analysis.document_id}...")
        print(f"   📊 System load: {current_load:.2f} | Budget: ${cost_budget:.2f} | Quality req: {quality_req:.2f}")
        
        # Start with recommended route
        candidate_route = analysis.recommended_route
        route_reason = f"Recommended for {analysis.complexity.value} complexity"
        
        # Apply dynamic adjustments
        
        # 1. System load considerations
        if current_load > self.system_load_threshold:
            if candidate_route == ProcessingRoute.ACCURATE:
                candidate_route = ProcessingRoute.BALANCED
                route_reason = "Downgraded due to high system load"
            elif candidate_route == ProcessingRoute.PREMIUM:
                candidate_route = ProcessingRoute.ACCURATE
                route_reason = "Downgraded due to high system load"
            print(f"   ⚠️ High system load detected, adjusting route")
        
        # 2. Budget constraints
        if analysis.estimated_cost > cost_budget:
            if candidate_route == ProcessingRoute.PREMIUM:
                candidate_route = ProcessingRoute.ACCURATE
            elif candidate_route == ProcessingRoute.ACCURATE:
                candidate_route = ProcessingRoute.BALANCED
            elif candidate_route == ProcessingRoute.BALANCED:
                candidate_route = ProcessingRoute.FAST
            route_reason = "Downgraded due to budget constraints"
            print(f"   💰 Budget constraint applied")
        
        # 3. Quality requirements
        if quality_req > analysis.expected_quality:
            if candidate_route == ProcessingRoute.FAST:
                candidate_route = ProcessingRoute.BALANCED
            elif candidate_route == ProcessingRoute.BALANCED:
                candidate_route = ProcessingRoute.ACCURATE
            elif candidate_route == ProcessingRoute.ACCURATE:
                candidate_route = ProcessingRoute.PREMIUM
            route_reason = "Upgraded to meet quality requirements"
            print(f"   🎯 Quality requirement upgrade applied")
        
        # 4. Historical performance optimization
        candidate_route = self._optimize_based_on_history(candidate_route, analysis)
        
        # Get updated estimates for final route
        final_cost, final_time, final_quality = self._get_route_estimates(candidate_route)
        
        state['selected_route'] = candidate_route
        state['route_reason'] = route_reason
        
        print(f"   ✅ Selected route: {candidate_route.value}")
        print(f"   📝 Reason: {route_reason}")
        print(f"   📊 Estimates: ${final_cost:.2f}, {final_time:.1f}s, quality {final_quality:.2f}")
        
        return state
    
    def _optimize_based_on_history(self, route: ProcessingRoute, analysis: DocumentAnalysis) -> ProcessingRoute:
        """Optimize route based on historical performance"""
        # Get historical performance for this route
        performance_data = self.route_performance.get(route, [])
        
        if len(performance_data) < 3:  # Not enough data
            return route
        
        # Calculate average efficiency (quality/cost ratio)
        avg_efficiency = sum(p['efficiency'] for p in performance_data[-5:]) / min(5, len(performance_data))
        
        # If efficiency is low, consider alternative routes
        if avg_efficiency < 0.3:
            alternatives = {
                ProcessingRoute.FAST: ProcessingRoute.BALANCED,
                ProcessingRoute.BALANCED: ProcessingRoute.ACCURATE,
                ProcessingRoute.ACCURATE: ProcessingRoute.BALANCED,  # Sometimes simpler is better
                ProcessingRoute.PREMIUM: ProcessingRoute.ACCURATE
            }
            
            if route in alternatives:
                print(f"   📈 Historical performance suggests trying {alternatives[route].value}")
                return alternatives[route]
        
        return route
    
    def _get_route_estimates(self, route: ProcessingRoute) -> tuple:
        """Get cost, time, and quality estimates for a route"""
        estimates = {
            ProcessingRoute.FAST: (0.3, 3.0, 0.75),
            ProcessingRoute.BALANCED: (1.0, 8.0, 0.88),
            ProcessingRoute.ACCURATE: (2.5, 15.0, 0.95),
            ProcessingRoute.PREMIUM: (5.0, 30.0, 0.98)
        }
        return estimates.get(route, (1.0, 8.0, 0.88))
    
    def record_performance(self, route: ProcessingRoute, cost: float, time: float, quality: float):
        """Record performance for future optimization"""
        efficiency = quality / max(cost, 0.01)
        performance = {
            'cost': cost,
            'time': time,
            'quality': quality,
            'efficiency': efficiency,
            'timestamp': datetime.now().isoformat()
        }
        
        self.route_performance[route].append(performance)
        
        # Keep only recent performance data (last 20 records)
        if len(self.route_performance[route]) > 20:
            self.route_performance[route] = self.route_performance[route][-20:]

# Global routing engine
routing_engine = RoutingEngine()

def make_routing_decision(state: SmartRoutingState) -> SmartRoutingState:
    """Wrapper function for routing engine"""
    return routing_engine.route_document(state)

print("🧠 Intelligent routing engine ready")
print("⚙️ Features: dynamic load balancing, budget optimization, quality assurance")
print("📈 Learns from historical performance to improve future routing decisions")

## Step 3: Route-Specific Processing Implementations

Each route has different processing implementations optimized for speed, cost, or accuracy.

In [ ]:
# Route-specific processing implementations
def process_fast_route(state: SmartRoutingState) -> SmartRoutingState:
    """Fast processing: Basic OCR + pattern matching"""
    doc_id = state['document_id']
    route = state['selected_route']
    
    print(f"⚡ FAST processing for {doc_id}...")
    
    # Start tracking
    routing_tracker.start_operation(doc_id, 'fast', 0.3)
    
    # Simulate fast processing
    time.sleep(0.5)  # Quick processing
    
    # Basic pattern-based extraction
    content = state['document_content']
    text = content.get('text', '')
    
    # Simple regex-based extraction
    import re
    amount_match = re.search(r'\$([0-9,]+\.?[0-9]*)', text)
    vendor_match = re.search(r'[Ff]rom:?\s*([A-Za-z\s]+)', text)
    
    extracted_amount = float(amount_match.group(1).replace(',', '')) if amount_match else content.get('amount', 0)
    extracted_vendor = vendor_match.group(1).strip() if vendor_match else content.get('vendor', 'Unknown')
    
    # Quality score based on extraction confidence
    quality_score = 0.75 if amount_match and vendor_match else 0.6
    
    results = {
        'route_used': 'fast',
        'processing_method': 'regex_extraction',
        'extracted_data': {
            'amount': extracted_amount,
            'vendor': extracted_vendor,
            'confidence': 'basic'
        },
        'validation_performed': False,
        'ai_analysis': False
    }
    
    # End tracking
    actual_time = routing_tracker.end_operation(doc_id, quality_score)
    
    state['processing_results'] = results
    state['quality_score'] = quality_score
    state['actual_time'] = actual_time
    state['actual_cost'] = routing_tracker.costs.get(doc_id, 0.3)
    
    print(f"   ✅ Fast processing completed: {quality_score:.2f} quality, ${state['actual_cost']:.2f} cost")
    
    return state

def process_balanced_route(state: SmartRoutingState) -> SmartRoutingState:
    """Balanced processing: OCR + LLM analysis"""
    doc_id = state['document_id']
    
    print(f"⚖️ BALANCED processing for {doc_id}...")
    
    # Start tracking
    routing_tracker.start_operation(doc_id, 'balanced', 1.0)
    
    # Simulate balanced processing
    time.sleep(1.2)  # Moderate processing time
    
    content = state['document_content']
    text = content.get('text', '')
    
    # Use LLM for extraction if available
    if OLLAMA_URL != 'http://XX.XX.XX.XX':
        prompt = f"""Extract key information from this invoice text:
{text[:500]}

Return JSON with: vendor, amount, currency, date, invoice_number"""
        
        llm_response = call_llm(prompt)
        
        # Try to parse JSON response
        try:
            extracted_data = json.loads(llm_response)
        except:
            # Fallback to text response
            extracted_data = {'raw_response': llm_response}
        
        quality_score = 0.88
    else:
        # Mock LLM extraction
        extracted_data = {
            'vendor': content.get('vendor', 'Unknown'),
            'amount': content.get('amount', 0),
            'currency': 'USD',
            'confidence': 'llm_analysis'
        }
        quality_score = 0.85
    
    # Add basic validation
    validation_checks = {
        'amount_reasonable': extracted_data.get('amount', 0) > 0,
        'vendor_present': bool(extracted_data.get('vendor')),
        'currency_valid': extracted_data.get('currency') in ['USD', 'EUR', 'GBP']
    }
    
    results = {
        'route_used': 'balanced',
        'processing_method': 'llm_extraction',
        'extracted_data': extracted_data,
        'validation_performed': True,
        'validation_results': validation_checks,
        'ai_analysis': True
    }
    
    # End tracking
    actual_time = routing_tracker.end_operation(doc_id, quality_score)
    
    state['processing_results'] = results
    state['quality_score'] = quality_score
    state['actual_time'] = actual_time
    state['actual_cost'] = routing_tracker.costs.get(doc_id, 1.0)
    
    print(f"   ✅ Balanced processing completed: {quality_score:.2f} quality, ${state['actual_cost']:.2f} cost")
    
    return state

def process_accurate_route(state: SmartRoutingState) -> SmartRoutingState:
    """Accurate processing: Multi-modal analysis + validation"""
    doc_id = state['document_id']
    
    print(f"🎯 ACCURATE processing for {doc_id}...")
    
    # Start tracking
    routing_tracker.start_operation(doc_id, 'accurate', 2.5)
    
    # Simulate accurate processing (longer time)
    time.sleep(2.0)
    
    content = state['document_content']
    
    # Multi-modal analysis (building on Session 2)
    extracted_data = {
        'vendor': content.get('vendor', 'Unknown'),
        'amount': content.get('amount', 0),
        'currency': 'USD'
    }
    
    # Enhanced analysis with multiple verification steps
    if OLLAMA_URL != 'http://XX.XX.XX.XX':
        # Multiple LLM calls for cross-validation
        prompts = [
            f"Extract the vendor name from: {content.get('text', '')[:200]}",
            f"What is the total amount in: {content.get('text', '')[:200]}",
            f"Verify the invoice details in: {content.get('text', '')[:200]}"
        ]
        
        llm_results = []
        for prompt in prompts:
            response = call_llm(prompt)
            llm_results.append(response)
        
        extracted_data['llm_cross_validation'] = llm_results
    
    # Comprehensive validation
    validation_checks = {
        'amount_reasonable': extracted_data.get('amount', 0) > 0,
        'vendor_present': bool(extracted_data.get('vendor')),
        'currency_valid': True,
        'cross_validation_consistent': True,  # Would check LLM consistency
        'format_compliance': True,
        'mathematical_accuracy': True  # Would verify calculations
    }
    
    # Confidence score based on validation results
    validation_success_rate = sum(validation_checks.values()) / len(validation_checks)
    quality_score = 0.9 + (validation_success_rate * 0.05)  # 0.90-0.95 range
    
    results = {
        'route_used': 'accurate',
        'processing_method': 'multimodal_analysis',
        'extracted_data': extracted_data,
        'validation_performed': True,
        'validation_results': validation_checks,
        'validation_success_rate': validation_success_rate,
        'cross_validation_performed': True,
        'ai_analysis': True
    }
    
    # End tracking
    actual_time = routing_tracker.end_operation(doc_id, quality_score)
    
    state['processing_results'] = results
    state['quality_score'] = quality_score
    state['actual_time'] = actual_time
    state['actual_cost'] = routing_tracker.costs.get(doc_id, 2.5)
    
    print(f"   ✅ Accurate processing completed: {quality_score:.2f} quality, ${state['actual_cost']:.2f} cost")
    
    return state

def process_premium_route(state: SmartRoutingState) -> SmartRoutingState:
    """Premium processing: Human-in-the-loop validation"""
    doc_id = state['document_id']
    
    print(f"👑 PREMIUM processing for {doc_id}...")
    
    # Start tracking
    routing_tracker.start_operation(doc_id, 'premium', 5.0)
    
    # Simulate premium processing (includes human review time)
    time.sleep(3.0)
    
    # First, run accurate processing
    content = state['document_content']
    
    # Simulate comprehensive AI analysis
    extracted_data = {
        'vendor': content.get('vendor', 'Unknown'),
        'amount': content.get('amount', 0),
        'currency': 'USD',
        'ai_confidence': 0.95
    }
    
    # Simulate human validation
    human_validation = {
        'human_reviewer': 'Expert-001',
        'review_time': '5 minutes',
        'corrections_made': 0,
        'final_approval': True,
        'human_confidence': 0.98
    }
    
    # Premium validation suite
    validation_checks = {
        'ai_validation': True,
        'human_validation': True,
        'cross_reference_check': True,
        'compliance_verification': True,
        'fraud_detection': True,
        'mathematical_verification': True,
        'format_compliance': True
    }
    
    quality_score = 0.98  # Highest quality
    
    results = {
        'route_used': 'premium',
        'processing_method': 'ai_plus_human_validation',
        'extracted_data': extracted_data,
        'human_validation': human_validation,
        'validation_performed': True,
        'validation_results': validation_checks,
        'ai_analysis': True,
        'human_review': True
    }
    
    # End tracking
    actual_time = routing_tracker.end_operation(doc_id, quality_score)
    
    state['processing_results'] = results
    state['quality_score'] = quality_score
    state['actual_time'] = actual_time
    state['actual_cost'] = routing_tracker.costs.get(doc_id, 5.0)
    
    print(f"   ✅ Premium processing completed: {quality_score:.2f} quality, ${state['actual_cost']:.2f} cost")
    
    return state

print("🛠️ Route-specific processing implementations ready")
print("⚡ Fast: Regex extraction (0.75 quality, $0.30)")
print("⚖️ Balanced: LLM analysis (0.88 quality, $1.00)")
print("🎯 Accurate: Multi-modal + validation (0.95 quality, $2.50)")
print("👑 Premium: AI + human review (0.98 quality, $5.00)")

## Step 4: Smart Routing Graph

Build the complete routing workflow that adapts based on document analysis and system conditions.

In [ ]:
# Dynamic routing function
def route_to_processor(state: SmartRoutingState) -> str:
    """Dynamic routing based on selected route"""
    route = state['selected_route']
    
    route_mapping = {
        ProcessingRoute.FAST: 'process_fast',
        ProcessingRoute.BALANCED: 'process_balanced',
        ProcessingRoute.ACCURATE: 'process_accurate',
        ProcessingRoute.PREMIUM: 'process_premium'
    }
    
    return route_mapping.get(route, 'process_balanced')

def calculate_efficiency_metrics(state: SmartRoutingState) -> SmartRoutingState:
    """Calculate final efficiency and performance metrics"""
    quality = state['quality_score']
    cost = state['actual_cost']
    time = state['actual_time']
    
    # Calculate efficiency metrics
    efficiency_score = quality / max(cost, 0.01)  # Quality per dollar
    speed_efficiency = quality / max(time, 0.1)   # Quality per second
    
    # Compare to expectations
    analysis = state['complexity_analysis']
    cost_efficiency = cost / max(analysis.estimated_cost, 0.01)
    time_efficiency = time / max(analysis.estimated_time, 0.1)
    quality_achievement = quality / max(analysis.expected_quality, 0.01)
    
    state['efficiency_score'] = efficiency_score
    
    # Store performance for routing engine learning
    route = state['selected_route']
    routing_engine.record_performance(route, cost, time, quality)
    
    print(f"📊 Final metrics for {state['document_id']}:")
    print(f"   💎 Quality achieved: {quality:.3f} ({quality_achievement:.1%} of expected)")
    print(f"   💰 Cost efficiency: {cost_efficiency:.1%} of estimated")
    print(f"   ⏱️ Time efficiency: {time_efficiency:.1%} of estimated")
    print(f"   🎯 Overall efficiency: {efficiency_score:.2f} (quality/cost ratio)")
    
    return state

# Build the smart routing graph
smart_routing_graph = StateGraph(SmartRoutingState)

# Add nodes
smart_routing_graph.add_node("analyze_complexity", analyze_document_complexity)
smart_routing_graph.add_node("make_routing_decision", make_routing_decision)
smart_routing_graph.add_node("process_fast", process_fast_route)
smart_routing_graph.add_node("process_balanced", process_balanced_route)
smart_routing_graph.add_node("process_accurate", process_accurate_route)
smart_routing_graph.add_node("process_premium", process_premium_route)
smart_routing_graph.add_node("calculate_metrics", calculate_efficiency_metrics)

# Set entry point
smart_routing_graph.set_entry_point("analyze_complexity")

# Add routing logic
smart_routing_graph.add_edge("analyze_complexity", "make_routing_decision")

# Dynamic routing to processors
smart_routing_graph.add_conditional_edges(
    "make_routing_decision",
    route_to_processor,
    {
        "process_fast": "process_fast",
        "process_balanced": "process_balanced",
        "process_accurate": "process_accurate",
        "process_premium": "process_premium"
    }
)

# All processors go to metrics calculation
smart_routing_graph.add_edge("process_fast", "calculate_metrics")
smart_routing_graph.add_edge("process_balanced", "calculate_metrics")
smart_routing_graph.add_edge("process_accurate", "calculate_metrics")
smart_routing_graph.add_edge("process_premium", "calculate_metrics")

# End workflow
smart_routing_graph.add_edge("calculate_metrics", END)

# Compile the graph
try:
    smart_routing_app = smart_routing_graph.compile()
    print("✅ Smart routing graph compiled successfully!")
except Exception as e:
    print(f"❌ Error compiling routing graph: {e}")

# Visualize the routing workflow
print("\n📊 Smart Routing Workflow:")
print("┌─────────────────┐")
print("│    Analyze      │")
print("│   Complexity    │")
print("└─────────┬───────┘")
print("          │")
print("          ▼")
print("┌─────────────────┐")
print("│  Smart Routing  │")
print("│    Decision     │")
print("└─────────┬───────┘")
print("          │")
print("    ┌─────┼─────┬─────┐")
print("    ▼     ▼     ▼     ▼")
print("┌─────┐ ┌───┐ ┌───┐ ┌────┐")
print("│Fast │ │Bal│ │Acc│ │Prem│")
print("│Route│ │Rte│ │Rte│ │Rte │")
print("└─────┘ └───┘ └───┘ └────┘")
print("    │     │     │     │")
print("    └─────┼─────┼─────┘")
print("          ▼")
print("  ┌─────────────────┐")
print("  │   Calculate     │")
print("  │   Efficiency    │")
  print("  └─────────────────┘")

## Step 5: Live Demonstration - Smart Routing in Action

Test the complete smart routing system with different document types and system conditions.

In [ ]:
# Create test documents with varying complexity
test_documents = [
    {
        'document_id': 'INV-SIMPLE-001',
        'document_content': {
            'text': 'Invoice from TechCorp. Amount: $1,200. Date: 2024-01-15',
            'image_quality': 0.9,
            'page_count': 1,
            'amount': 1200.0,
            'vendor': 'TechCorp'
        },
        'system_conditions': {
            'current_load': 0.3,
            'cost_budget': 2.0,
            'quality_requirement': 0.8
        }
    },
    {
        'document_id': 'INV-MODERATE-002',
        'document_content': {
            'text': 'Invoice from Global Suppliers Inc. Complex multi-line items. Total amount: $8,500',
            'image_quality': 0.7,
            'page_count': 2,
            'amount': 8500.0,
            'vendor': 'Global Suppliers Inc.'
        },
        'system_conditions': {
            'current_load': 0.6,
            'cost_budget': 1.5,
            'quality_requirement': 0.9
        }
    },
    {
        'document_id': 'INV-COMPLEX-003',
        'document_content': {
            'text': 'Multi-language invoice from International Corp. Poor scan quality. Amount: $15,000',
            'image_quality': 0.4,
            'page_count': 3,
            'amount': 15000.0,
            'vendor': 'International Corp'
        },
        'system_conditions': {
            'current_load': 0.9,
            'cost_budget': 3.0,
            'quality_requirement': 0.95
        }
    },
    {
        'document_id': 'INV-CRITICAL-004',
        'document_content': {
            'text': 'High-value contract invoice from Trusted Partners LLC. Amount: $75,000',
            'image_quality': 0.8,
            'page_count': 1,
            'amount': 75000.0,
            'vendor': 'Trusted Partners LLC'
        },
        'system_conditions': {
            'current_load': 0.4,
            'cost_budget': 10.0,
            'quality_requirement': 0.98
        }
    }
]

def create_routing_state(doc_data: Dict) -> SmartRoutingState:
    """Create initial routing state from test data"""
    return SmartRoutingState(
        document_id=doc_data['document_id'],
        document_content=doc_data['document_content'],
        complexity_analysis=None,
        selected_route=None,
        route_reason=None,
        processing_results=None,
        quality_score=None,
        actual_cost=None,
        actual_time=None,
        efficiency_score=None,
        current_load=doc_data['system_conditions']['current_load'],
        cost_budget=doc_data['system_conditions']['cost_budget'],
        quality_requirement=doc_data['system_conditions']['quality_requirement']
    )

print("🚀 SMART ROUTING DEMONSTRATION")
print("=" * 60)
print("Testing intelligent routing with different document types and system conditions")

# Process each test document
results_summary = []

for i, doc_data in enumerate(test_documents, 1):
    print(f"\n📄 TEST {i}: {doc_data['document_id']}")
    print("-" * 50)
    
    # Create initial state
    initial_state = create_routing_state(doc_data)
    
    # Track memory usage
    memory_before = get_memory_usage()
    start_time = time.time()
    
    try:
        # Process through smart routing
        if 'smart_routing_app' in globals():
            result = smart_routing_app.invoke(initial_state)
            
            # Calculate total metrics
            total_time = time.time() - start_time
            memory_after = get_memory_usage()
            
            # Extract results
            complexity = result['complexity_analysis']
            route_used = result['selected_route']
            quality = result['quality_score']
            cost = result['actual_cost']
            efficiency = result['efficiency_score']
            
            # Display results
            print(f"\n📊 RESULTS:")
            print(f"   🔍 Complexity: {complexity.complexity.value} (confidence: {complexity.confidence:.2f})")
            print(f"   🛤️ Route used: {route_used.value}")
            print(f"   📝 Reason: {result['route_reason']}")
            print(f"   💎 Quality achieved: {quality:.3f}")
            print(f"   💰 Cost: ${cost:.2f}")
            print(f"   ⏱️ Time: {result['actual_time']:.1f}s")
            print(f"   🎯 Efficiency: {efficiency:.2f}")
            print(f"   💾 Memory impact: +{memory_after - memory_before:.1f}MB")
            
            # Store for summary
            results_summary.append({
                'document_id': doc_data['document_id'],
                'complexity': complexity.complexity.value,
                'route': route_used.value,
                'quality': quality,
                'cost': cost,
                'time': result['actual_time'],
                'efficiency': efficiency
            })
            
        else:
            print("❌ Smart routing application not available")
            
    except Exception as e:
        print(f"❌ Processing failed: {e}")
        results_summary.append({
            'document_id': doc_data['document_id'],
            'error': str(e)
        })

# Display summary
if results_summary:
    print(f"\n\n📈 SMART ROUTING SUMMARY")
    print("=" * 60)
    
    for result in results_summary:
        if 'error' not in result:
            print(f"{result['document_id']:20} | {result['complexity']:8} | {result['route']:8} | {result['quality']:.2f} | ${result['cost']:.2f} | {result['efficiency']:.1f}")
    
    # Calculate aggregate metrics
    valid_results = [r for r in results_summary if 'error' not in r]
    if valid_results:
        total_cost = sum(r['cost'] for r in valid_results)
        avg_quality = sum(r['quality'] for r in valid_results) / len(valid_results)
        avg_efficiency = sum(r['efficiency'] for r in valid_results) / len(valid_results)
        
        print(f"\n📊 AGGREGATE METRICS:")
        print(f"   Documents processed: {len(valid_results)}")
        print(f"   Total cost: ${total_cost:.2f}")
        print(f"   Average quality: {avg_quality:.3f}")
        print(f"   Average efficiency: {avg_efficiency:.2f}")
        
        # Route distribution
        route_counts = {}
        for r in valid_results:
            route_counts[r['route']] = route_counts.get(r['route'], 0) + 1
        
        print(f"   Route distribution: {dict(route_counts)}")

print(f"\n💡 INSIGHTS:")
print(f"   🧠 Smart routing automatically adapts to document complexity")
print(f"   ⚖️ Balances cost, quality, and system load constraints")
print(f"   📈 Higher complexity documents → more accurate (expensive) routes")
print(f"   🎯 System optimizes for efficiency (quality/cost ratio)")

## Step 6: Routing Analytics and Optimization

Analyze routing performance to optimize future decisions and demonstrate cost savings.

In [ ]:
# Get routing analytics
print("📊 ROUTING ANALYTICS & OPTIMIZATION")
print("=" * 60)

analytics = routing_tracker.get_routing_analytics()

if analytics.get('status') != 'no_data':
    route_stats = analytics['route_stats']
    
    print(f"\n📈 ROUTE PERFORMANCE ANALYSIS:")
    print(f"{'Route':<12} {'Count':<6} {'Avg Time':<10} {'Avg Cost':<10} {'Avg Quality':<12} {'Efficiency':<10}")
    print("-" * 70)
    
    for route, stats in route_stats.items():
        print(f"{route:<12} {stats['count']:<6} {stats['avg_time']:<10.1f} ${stats['avg_cost']:<9.2f} {stats['avg_quality']:<12.3f} {stats['efficiency']:<10.2f}")
    
    print(f"\n💰 COST ANALYSIS:")
    print(f"   Total operations: {analytics['total_operations']}")
    print(f"   Total cost: ${analytics['total_cost']:.2f}")
    print(f"   Average cost per document: ${analytics['total_cost'] / analytics['total_operations']:.2f}")
    print(f"   Average quality: {analytics['avg_quality']:.3f}")
    
    # Calculate cost savings vs always using premium route
    premium_cost_all = analytics['total_operations'] * 5.0  # $5 per premium processing
    actual_cost = analytics['total_cost']
    cost_savings = premium_cost_all - actual_cost
    savings_percentage = (cost_savings / premium_cost_all) * 100
    
    print(f"\n💡 COST OPTIMIZATION BENEFITS:")
    print(f"   Cost if all premium: ${premium_cost_all:.2f}")
    print(f"   Actual smart routing cost: ${actual_cost:.2f}")
    print(f"   Cost savings: ${cost_savings:.2f} ({savings_percentage:.1f}%)")
    
    # Find most efficient route
    best_efficiency_route = max(route_stats.items(), key=lambda x: x[1]['efficiency'])
    print(f"   Most efficient route: {best_efficiency_route[0]} (efficiency: {best_efficiency_route[1]['efficiency']:.2f})")
    
    # Recommendations for optimization
    print(f"\n🎯 OPTIMIZATION RECOMMENDATIONS:")
    
    for route, stats in route_stats.items():
        if stats['efficiency'] < 0.3:
            print(f"   ⚠️ {route} route showing low efficiency - consider tuning")
        elif stats['efficiency'] > 1.0:
            print(f"   ✅ {route} route performing excellently - maintain current approach")
    
    # System load recommendations
    if analytics['total_operations'] > 0:
        print(f"   📊 For future scaling:")
        print(f"      • Monitor system load to prevent routing downgrades")
        print(f"      • Consider dedicated processing pools for each route")
        print(f"      • Implement predictive cost budgeting")

else:
    print("⚠️ No routing data available for analysis")

# Demonstrate different system scenarios
print(f"\n\n🔬 SCENARIO ANALYSIS")
print("=" * 40)

scenarios = [
    {
        'name': 'High Load + Budget Constraints',
        'conditions': {'current_load': 0.9, 'cost_budget': 0.5, 'quality_requirement': 0.8},
        'expected_behavior': 'Route to fast processing despite complexity'
    },
    {
        'name': 'Low Load + High Quality Needs',
        'conditions': {'current_load': 0.2, 'cost_budget': 10.0, 'quality_requirement': 0.97},
        'expected_behavior': 'Route to premium processing for quality'
    },
    {
        'name': 'Balanced Operations',
        'conditions': {'current_load': 0.5, 'cost_budget': 2.0, 'quality_requirement': 0.85},
        'expected_behavior': 'Route based on document complexity'
    }
]

print("\n🎛️ How routing adapts to different conditions:")
for scenario in scenarios:
    print(f"\n   {scenario['name']}:")
    conditions = scenario['conditions']
    print(f"      Load: {conditions['current_load']:.1f} | Budget: ${conditions['cost_budget']:.1f} | Quality: {conditions['quality_requirement']:.2f}")
    print(f"      Expected: {scenario['expected_behavior']}")

# Performance comparison: Smart vs Fixed routing
print(f"\n\n⚖️ SMART ROUTING vs FIXED ROUTING COMPARISON")
print("=" * 60)

if analytics.get('status') != 'no_data':
    # Calculate what fixed routing would cost
    operations_count = analytics['total_operations']
    
    fixed_scenarios = {
        'Always Fast': operations_count * 0.3,
        'Always Balanced': operations_count * 1.0,
        'Always Accurate': operations_count * 2.5,
        'Always Premium': operations_count * 5.0
    }
    
    smart_cost = analytics['total_cost']
    smart_quality = analytics['avg_quality']
    
    print(f"\n💰 Cost Comparison (for {operations_count} documents):")
    print(f"{'Strategy':<15} {'Total Cost':<12} {'vs Smart':<12} {'Quality':<10}")
    print("-" * 50)
    
    quality_estimates = {'Always Fast': 0.75, 'Always Balanced': 0.88, 'Always Accurate': 0.95, 'Always Premium': 0.98}
    
    print(f"{'Smart Routing':<15} ${smart_cost:<11.2f} {'baseline':<12} {smart_quality:<10.3f}")
    
    for strategy, cost in fixed_scenarios.items():
        difference = cost - smart_cost
        diff_sign = "+" if difference > 0 else ""
        quality_est = quality_estimates[strategy]
        print(f"{strategy:<15} ${cost:<11.2f} {diff_sign}${difference:<11.2f} {quality_est:<10.3f}")
    
    # Find the most cost-effective fixed strategy
    best_fixed = min(fixed_scenarios.items(), key=lambda x: abs(x[1] - smart_cost))
    print(f"\n🎯 Closest fixed strategy: {best_fixed[0]} (${best_fixed[1]:.2f})")
    print(f"   Smart routing advantage: ${abs(best_fixed[1] - smart_cost):.2f} savings")

print(f"\n🏆 KEY BENEFITS OF SMART ROUTING:")
print(f"   🧠 Adaptive decision making based on document complexity")
print(f"   💰 Cost optimization without sacrificing necessary quality")
print(f"   ⚖️ Dynamic load balancing to maintain system performance")
print(f"   📈 Continuous learning from routing performance")
print(f"   🎯 Business-aware routing (cost budgets, quality requirements)")

## Key Learnings

### Evolution Through the Sessions:

1. **Session 1: Concurrent Processing Foundation**
   - Parallel execution of independent operations
   - 3-5x speedup through smart dependency management
   - Production-ready performance tracking

2. **Session 2: Multimodal Intelligence**
   - Text + Image + Structured data processing
   - Concurrent modality processing
   - Rich document understanding capabilities

3. **Session 3: Smart Routing & Optimization**
   - **Intelligent decision making** based on document complexity
   - **Cost-aware routing** that balances quality and budget
   - **Adaptive optimization** that learns from performance
   - **Business-aligned processing** with configurable requirements

### Smart Routing Benefits:

**Cost Optimization:**
- 40-80% cost savings vs always using premium processing
- Quality-aware routing ensures minimum standards are met
- Dynamic budget management prevents cost overruns

**Performance Intelligence:**
- Document complexity analysis guides processing decisions
- System load balancing prevents performance degradation
- Historical performance learning improves routing over time

**Business Alignment:**
- Configurable quality requirements per use case
- Cost budget constraints for financial control
- Route-specific processing optimized for different needs

### Processing Routes Comparison:

| Route | Cost | Time | Quality | Use Case |
|-------|------|------|---------|----------|
| **Fast** | $0.30 | 3s | 75% | Simple documents, high volume |
| **Balanced** | $1.00 | 8s | 88% | Standard processing |
| **Accurate** | $2.50 | 15s | 95% | Complex documents, validation needed |
| **Premium** | $5.00 | 30s | 98% | High-value, critical documents |

### Technical Architecture:

- **Complexity Analysis**: Multi-factor document assessment
- **Routing Engine**: Business-rule aware decision making
- **Route Processors**: Optimized implementations per quality/cost target
- **Performance Analytics**: Continuous optimization feedback loop

### Production Considerations:

**Monitoring:**
- Track routing decisions and outcomes
- Monitor cost vs quality trade-offs
- Analyze efficiency trends over time

**Optimization:**
- Adjust complexity thresholds based on performance
- Fine-tune cost estimates for each route
- Implement A/B testing for routing strategies

**Scaling:**
- Separate processing pools for each route
- Dynamic resource allocation based on route demand
- Predictive routing based on document batches

### Real-World Impact:

- **Enterprise Scale**: Process thousands of documents cost-effectively
- **Quality Assurance**: Meet business requirements without over-processing
- **Cost Control**: Predictable, optimized processing costs
- **Adaptive Systems**: Self-improving AI that gets better over time

### What's Next:

Session 4 will add **Multi-Document Batch Processing**:
- Scale smart routing to handle document batches
- Cross-document pattern recognition
- Batch-level optimization strategies
- Enterprise-grade throughput capabilities

This completes the intelligence layer: we now have **concurrent processing** + **multimodal understanding** + **smart optimization** - the complete foundation for production AI document processing systems.